In [1]:
import json
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

In [27]:
Institutions = "resources/cc_institution_details.csv"
Institutions_db = pd.read_csv(Institutions,encoding="cp1252")

In [28]:
Institutions_db.head()

,unitid,chronname,city,state,level,control,basic,hbcu,flagship,long_x,...,vsa_grad_after6_transfer,vsa_grad_elsewhere_after6_transfer,vsa_enroll_after6_transfer,vsa_enroll_elsewhere_after6_transfer,similar,state_sector_ct,carnegie_ct,counted_pct,nicknames,cohort_size
0,100654,Alabama A&M University,Normal,Alabama,4-year,Public,Masters Colleges and Universities--larger prog...,X,NaN,-86.568502,...,36.4,5.6,17.2,11.1,232937|100724|405997|113607|139533|144005|2285...,13,386,99.7|07,NaN,882.0
1,100663,University of Alabama at Birmingham,Birmingham,Alabama,4-year,Public,Research Universities--very high research acti...,NaN,NaN,-86.809170,...,NaN,NaN,NaN,NaN,196060|180461|201885|145600|209542|236939|1268...,13,106,56.0|07,UAB,1376.0
2,100690,Amridge University,Montgomery,Alabama,4-year,Private not-for-profit,Baccalaureate Colleges--Arts & Sciences,NaN,NaN,-86.174010,...,NaN,NaN,NaN,NaN,217925|441511|205124|247825|197647|221856|1353...,16,252,100.0|07,NaN,3.0
3,100706,University of Alabama at Huntsville,Huntsville,Alabama,4-year,Public,Research Universities--very high research acti...,NaN,NaN,-86.638420,...,0.0,0.0,0.0,0.0,232186|133881|196103|196413|207388|171128|1900...,13,106,43.1|07,UAH,759.0
4,100724,Alabama State University,Montgomery,Alabama,4-year,Public,Masters Colleges and Universities--larger prog...,X,NaN,-86.295677,...,NaN,NaN,NaN,NaN,100654|232937|242617|243197|144005|241739|2354...,13,386,88.0|07,ASU,1351.0


In [29]:
len(Institutions_db)

3798

In [30]:
MF = Institutions_db.reset_index()

In [71]:
Inst = MF[["unitid","chronname","city","state","level","control","student_count","med_sat_value","aid_value","grad_100_value","grad_150_value"]]

In [72]:
Inst

,unitid,chronname,city,state,level,control,student_count,med_sat_value,aid_value,grad_100_value,grad_150_value
0,100654,Alabama A&M University,Normal,Alabama,4-year,Public,4051,823.0,7142.0,10.0,29.1
1,100663,University of Alabama at Birmingham,Birmingham,Alabama,4-year,Public,11502,1146.0,6088.0,29.4,53.5
2,100690,Amridge University,Montgomery,Alabama,4-year,Private not-for-profit,322,NaN,2540.0,0.0,66.7
3,100706,University of Alabama at Huntsville,Huntsville,Alabama,4-year,Public,5696,1180.0,6647.0,16.5,48.4
4,100724,Alabama State University,Montgomery,Alabama,4-year,Public,5356,830.0,7256.0,8.8,25.2
...,...,...,...,...,...,...,...,...,...,...,...
3793,461528,Grace College of Divinity,Fayetteville,North Carolina,4-year,Private not-for-profit,110,NaN,4860.0,0.0,0.0
3794,462354,John Paul the Great Catholic University,Escondido,California,4-year,Private not-for-profit,140,1069.0,9395.0,51.5,51.5
3795,466921,Chamberlain College of Nursing-Missouri,St. Louis,Missouri,4-year,Private for-profit,488,NaN,3291.0,37.5,37.5
3796,474881,Minneapolis Media Institute,Edina,Minnesota,2-year,Private for-profit,147,NaN,3434.0,42.1,55.9


In [73]:
Inst.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3798 entries, 0 to 3797
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   unitid          3798 non-null   int64  
 1   chronname       3798 non-null   object 
 2   city            3798 non-null   object 
 3   state           3798 non-null   object 
 4   level           3798 non-null   object 
 5   control         3798 non-null   object 
 6   student_count   3798 non-null   int64  
 7   med_sat_value   1337 non-null   float64
 8   aid_value       3797 non-null   float64
 9   grad_100_value  3467 non-null   float64
 10  grad_150_value  3467 non-null   float64
dtypes: float64(4), int64(2), object(5)
memory usage: 326.5+ KB


In [75]:
Inst = Inst.dropna()

In [76]:
Inst.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1319 entries, 0 to 3794
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   unitid          1319 non-null   int64  
 1   chronname       1319 non-null   object 
 2   city            1319 non-null   object 
 3   state           1319 non-null   object 
 4   level           1319 non-null   object 
 5   control         1319 non-null   object 
 6   student_count   1319 non-null   int64  
 7   med_sat_value   1319 non-null   float64
 8   aid_value       1319 non-null   float64
 9   grad_100_value  1319 non-null   float64
 10  grad_150_value  1319 non-null   float64
dtypes: float64(4), int64(2), object(5)
memory usage: 123.7+ KB


In [77]:
Inst

,unitid,chronname,city,state,level,control,student_count,med_sat_value,aid_value,grad_100_value,grad_150_value
0,100654,Alabama A&M University,Normal,Alabama,4-year,Public,4051,823.0,7142.0,10.0,29.1
1,100663,University of Alabama at Birmingham,Birmingham,Alabama,4-year,Public,11502,1146.0,6088.0,29.4,53.5
3,100706,University of Alabama at Huntsville,Huntsville,Alabama,4-year,Public,5696,1180.0,6647.0,16.5,48.4
4,100724,Alabama State University,Montgomery,Alabama,4-year,Public,5356,830.0,7256.0,8.8,25.2
5,100751,University of Alabama at Tuscaloosa,Tuscaloosa,Alabama,4-year,Public,29440,1171.0,10390.0,42.7,66.7
...,...,...,...,...,...,...,...,...,...,...,...
3572,446048,Ave Maria University,Ave Maria,Florida,4-year,Private not-for-profit,906,1070.0,15463.0,54.7,57.7
3639,448840,University of South Florida at St. Petersburg,St. Petersburg,Florida,4-year,Public,4025,1070.0,5005.0,22.0,42.7
3651,449764,Visible School,Memphis,Tennessee,4-year,Private not-for-profit,118,1070.0,4958.0,0.0,0.0
3717,454184,King's College (N.Y.),New York,New York,4-year,Private not-for-profit,516,1160.0,19698.0,56.8,61.4


In [47]:
 Inst.groupby(["level"]).count()

,unitid,chronname,city,state,control,student_count,med_sat_value
level,,,,,,,
2-year,1459,1459,1459,1459,1459,1459,22
4-year,2339,2339,2339,2339,2339,2339,1315


In [7]:
len(Institutions_db)

3798

In [52]:
Alabama = Inst.loc[(Inst["state"] == "Alabama")]

In [53]:
Alabama

,unitid,chronname,city,state,level,control,student_count,med_sat_value
0,100654,Alabama A&M University,Normal,Alabama,4-year,Public,4051,823.0
1,100663,University of Alabama at Birmingham,Birmingham,Alabama,4-year,Public,11502,1146.0
2,100690,Amridge University,Montgomery,Alabama,4-year,Private not-for-profit,322,NaN
3,100706,University of Alabama at Huntsville,Huntsville,Alabama,4-year,Public,5696,1180.0
4,100724,Alabama State University,Montgomery,Alabama,4-year,Public,5356,830.0
...,...,...,...,...,...,...,...,...
3673,450377,Strayer University-Alabama,Birmingham,Alabama,4-year,Private for-profit,390,NaN
3683,450933,Columbia Southern University,Orange Beach,Alabama,4-year,Private for-profit,14853,NaN
3697,451945,ITT Technical Institute-Madison,Madison,Alabama,4-year,Private for-profit,372,NaN
3706,452115,Virginia College-Montgomery,Montgomery,Alabama,2-year,Private for-profit,656,NaN


In [78]:
Howmanystates = Inst.groupby(["state"]).count()

In [79]:
Howmanystates

,unitid,chronname,city,level,control,student_count,med_sat_value,aid_value,grad_100_value,grad_150_value
state,,,,,,,,,,
Alabama,23,23,23,23,23,23,23,23,23,23
Alaska,1,1,1,1,1,1,1,1,1,1
Arizona,6,6,6,6,6,6,6,6,6,6
Arkansas,18,18,18,18,18,18,18,18,18,18
California,81,81,81,81,81,81,81,81,81,81
Colorado,15,15,15,15,15,15,15,15,15,15
Connecticut,14,14,14,14,14,14,14,14,14,14
Delaware,3,3,3,3,3,3,3,3,3,3
District of Columbia,6,6,6,6,6,6,6,6,6,6


In [85]:
Inst.to_csv("resources/Inst.csv")

In [57]:
len(Howmanystates)

51

In [58]:
InstSort = Inst.sort_values("control")

In [59]:
InstSort

,unitid,chronname,city,state,level,control,student_count,med_sat_value
3797,475291,Brown College-Brooklyn Center,Brooklyn Center,Minnesota,4-year,Private for-profit,144,NaN
3142,373085,Brown Mackie College - Quad Cities,Bettendorf,Iowa,2-year,Private for-profit,364,NaN
3141,373058,Vatterott College - Des Moines,Des Moines,Iowa,2-year,Private for-profit,239,NaN
3139,372921,Medix College,Smyrna,Georgia,2-year,Private for-profit,617,NaN
3138,372772,Heritage Institute - Jacksonville,Jacksonville,Florida,2-year,Private for-profit,279,NaN
...,...,...,...,...,...,...,...,...
1113,162104,Cecil College,North East,Maryland,2-year,Public,2604,NaN
1114,162122,College of Southern Maryland,La Plata,Maryland,2-year,Public,8781,NaN
2472,217882,Clemson University,Clemson,South Carolina,4-year,Public,16931,1259.0
2485,218140,Horry-Georgetown Technical College,Conway,South Carolina,2-year,Public,7660,NaN


In [86]:
Grads = "resources/cc_institution_grads.csv"
Grads_db = pd.read_csv(Grads)

In [87]:
Grads_db

,unitid,year,gender,race,cohort,grad_cohort,grad_100,grad_150,grad_100_rate,grad_150_rate
0,100760,2011,B,X,2y all,446.0,73.0,105.0,16.4,23.5
1,100760,2011,M,X,2y all,185.0,NaN,40.0,NaN,21.6
2,100760,2011,F,X,2y all,261.0,NaN,65.0,NaN,24.9
3,100760,2011,B,W,2y all,348.0,NaN,86.0,NaN,24.7
4,100760,2011,M,W,2y all,162.0,NaN,35.0,NaN,21.6
...,...,...,...,...,...,...,...,...,...,...
1302097,168591,2002,F,Ai,4y other,NaN,NaN,NaN,NaN,NaN
1302098,168740,2002,F,Ai,4y other,0.0,NaN,0.0,NaN,NaN
1302099,169716,2002,F,Ai,4y other,NaN,NaN,NaN,NaN,NaN
1302100,170082,2002,F,Ai,4y other,NaN,NaN,NaN,NaN,NaN
